In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load the data
data_new = pd.read_csv("/content/gaussian_spectra_1545.997.csv")

# Rename the columns
data_new.columns = ['Wavelength (nm)', 'Power (a.u.)']

# Sort the data by wavelength
data_new = data_new.sort_values(by='Wavelength (nm)').reset_index(drop=True)

# Normalize the power data
data_new['Normalized Power (a.u.)'] = (data_new['Power (a.u.)'] - data_new['Power (a.u.)'].min()) / (data_new['Power (a.u.)'].max() - data_new['Power (a.u.)'].min())

# Find peaks
peaks_new, _ = find_peaks(data_new['Normalized Power (a.u.)'], prominence=0.1)

# Peak wavelengths, power and FWHM
peak_data_new = []

# Loop through each peak and calculate FWHM
for peak in peaks_new:
    # Half maximum
    half_max = (data_new['Normalized Power (a.u.)'][peak] / 2)

    # Data on the left and right of the peak
    left_data = data_new[data_new['Wavelength (nm)'] < data_new['Wavelength (nm)'][peak]]
    right_data = data_new[data_new['Wavelength (nm)'] > data_new['Wavelength (nm)'][peak]]

    # Find the points in the data that are closest to the half maximum
    left_half_max = left_data.iloc[(left_data['Normalized Power (a.u.)'] - half_max).abs().argsort()[:1]]
    right_half_max = right_data.iloc[(right_data['Normalized Power (a.u.)'] - half_max).abs().argsort()[:1]]

    # Calculate FWHM
    fwhm = right_half_max['Wavelength (nm)'].values[0] - left_half_max['Wavelength (nm)'].values[0]

    # Store peak data
    peak_data_new.append({
        'Peak Wavelength (nm)': data_new['Wavelength (nm)'][peak],
        'Peak Power (a.u.)': data_new['Power (a.u.)'][peak],
        'FWHM (nm)': fwhm
    })

# Convert to DataFrame
peak_data_new = pd.DataFrame(peak_data_new)

# Normalize the peak power values
peak_data_new['Normalized Peak Power (a.u.)'] = (peak_data_new['Peak Power (a.u.)'] - data_new['Power (a.u.)'].min()) / (data_new['Power (a.u.)'].max() - data_new['Power (a.u.)'].min())

# Plotting the figure with clearer markings
plt.figure(figsize=(12, 8))
plt.plot(data_new['Wavelength (nm)'], data_new['Normalized Power (a.u.)'], label='Normalized Power', color='blue')
plt.scatter(peak_data_new['Peak Wavelength (nm)'], peak_data_new['Normalized Peak Power (a.u.)'], color='red', s=100, zorder=5)

# Draw vertical lines at the peak wavelengths
for i in range(peak_data_new.shape[0]):
    plt.axvline(peak_data_new.iloc[i, 0], linestyle='--', color='red')

plt.xlabel('Wavelength (nm)', fontsize=14)
plt.ylabel('Normalized Power (a.u.)', fontsize=14)
plt.title('Normalized Power vs Wavelength', fontsize=16)
plt.grid(True)

# Annotate peaks
for i in range(peak_data_new.shape[0]):
    plt.annotate(f"Peak {i+1}\nWavelength: {peak_data_new.iloc[i, 0]:.4f} nm\nPower: {peak_data_new.iloc[i, 1]:.6f} a.u.\nFWHM: {peak_data_new.iloc[i, 2]:.4f} nm",
                 (peak_data_new.iloc[i, 0], peak_data_new.iloc[i, 3]),
                 textcoords="offset points",
                 xytext=(-15,15),
                 ha='center',
                 fontsize=10,
                 color='darkred',
                 bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1))

plt.legend()
plt.show()

# Print out the peak data
for i in range(peak_data_new.shape[0]):
    print(f"Peak {i+1}")
    print(f"Wavelength: {peak_data_new.iloc[i, 0]:.4f} nm")
    print(f"Normalized Power: {peak_data_new.iloc[i, 3]:.6f} a.u.")
    print(f"FWHM: {peak_data_new.iloc[i, 2]:.4f} nm")
    print("\n")
